In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import os, sys, getpass
user = getpass.getuser()
sys.path.append(f'/home/{user}/codes/hpe_library/')
from lib_import import *
from my_utils import *
os.chdir('/home/hrai/codes/MotionBERT')

from lib.data.datareader_total import DataReaderTotal
from lib.data.datareader_h36m import DataReaderH36M
from lib.data.datareader_fit3d import DataReaderFIT3D
from lib.utils.args import get_opts_args, get_opt_args_from_model_name
from lib.model.load_model import load_model
from lib.model.load_dataset import load_dataset
from lib.data.dataset_motion_3d import MotionDataset3DTotal
from lib.model.evaluation import *

In [7]:
import requests
from tqdm import tqdm
import copy

# Notion API 키 및 데이터베이스 ID 설정
notion = readpkl('notion.pkl')
NOTION_API_KEY = notion['NOTION_API_KEY']
DATABASE_ID = notion['DATABASE_ID']

# 헤더 설정
headers = {
    "Authorization": f"Bearer {NOTION_API_KEY}",
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28"
}

def get_all_database_pages():
    url = f"https://api.notion.com/v1/databases/{DATABASE_ID}/query"
    all_pages = []
    has_more = True
    start_cursor = None
    
    while has_more:
        if start_cursor:
            data = {"start_cursor": start_cursor}
        else:
            data = {}
        
        response = requests.post(url, headers=headers, json=data)
        response_data = response.json()
        
        all_pages.extend(response_data["results"])
        has_more = response_data.get("has_more", False)
        start_cursor = response_data.get("next_cursor", None)
    print(f"Total pages: {len(all_pages)}")
    return all_pages

pages = get_all_database_pages()

pages_dict = {}
for page in pages:
    pages_dict[page["id"]] = page
    
model_dict = {}
for page in pages:
    page_id = page["id"]
    property = page["properties"]
    try:
        model_name = property["Model"]['select']['name']
    except: continue
    if model_name not in model_dict:
        model_dict[model_name] = []
    model_dict[model_name].append(page_id)

Total pages: 210


In [3]:
result_dict = readpkl('result_dict.pkl')

In [6]:
subset_dict = {}
for checkpoint in result_dict.keys():
    if 'cam_no_factor' not in checkpoint: continue
    print(f'\n{checkpoint}')
    model_ids = model_dict[checkpoint]
    for model_id in model_ids:
        page = pages_dict[model_id]
        train = page['properties']['Train dataset']['select'] 
        test = page['properties']['Test dataset']['select']
        # filtering - parent
        if test == None: continue # parent
        
        # get child page's info
        child_page = pages_dict[model_id]
        child_properties = child_page['properties']
        if len(child_properties['Name']['title']) == 0: continue
        child_title = child_properties['Name']['title'][0]['text']['content']
        if child_title in result_dict[checkpoint].keys():
            mpjpe = child_properties['MPJPE_prev']['number']
            p_mpjpe = child_properties['P-MPJPE_prev']['number']
            if mpjpe == None: mpjpe = -1
            if p_mpjpe == None: p_mpjpe = -1
            if len(result_dict[checkpoint][child_title].keys()) >0 :
                #print(f'\t{child_title}')
                e1 = result_dict[checkpoint][child_title]["e1"]
                e2 = result_dict[checkpoint][child_title]["e2"]
                if e1 == None: e1 = -1
                if e2 == None: e2 = -1
            else:
                e1 = -1
                e2 = -1
            if e1 > 0 and mpjpe > 0:
                if e1 - mpjpe > 1: mark = '⬆️'
                elif e1 - mpjpe < -1: mark = '⬇️'
                else: mark = '✅'
            else:
                mark = '❌'
            print(f'{mark}\t{e1:.2f} {e2:.2f} <- {mpjpe:.2f} {p_mpjpe:.2f}\t{child_title}')
            if child_title not in subset_dict: subset_dict[child_title] = []
            subset_dict[child_title].append(mark)


MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_same_z_ts_s4710
✅	159.42 117.13 <- 159.63 117.19	3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_ALL_TRAIN
⬇️	167.25 121.53 <- 288.42 154.17	3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_TS1_6
✅	113.02 84.69 <- 113.02 84.69	H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z
✅	108.62 79.20 <- 108.62 79.20	H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TR_S1_TS_S5678
✅	30.49 14.50 <- 30.49 14.50	FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TS_S4710

MB_train_fit3d_gt_cam_no_factor_with_canonical2_tr_s03
❌	186.13 125.26 <- -1.00 -1.00	3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN
❌	41.92 26.26 <- -1.00 -1.00	FIT3D-GT-CAM_NO_FACTOR-TR_S03
❌	133.72 100.37 <- -1.00 -1.00	H36M-GT-CAM_NO_FACTOR

MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_tr_s03
⬆️	187.67 132.01 <- 177.80 124.67	3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_ALL_TRAIN
⬆️	166.51 120.04 <- 153.08 1

In [9]:
for subset in natsorted(subset_dict):
    print(subset, subset_dict[subset])

3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_FIXED_DIST_5-TEST_ALL_TRAIN ['❌', '❌']
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_FIXED_DIST_5-TEST_TS1_6 ['❌', '❌']
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_ALL_TRAIN ['✅', '⬆️', '⬇️', '✅', '⬆️', '⬇️', '⬇️', '⬇️']
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_TS1_6 ['⬇️', '⬆️', '⬆️', '⬆️', '⬇️', '⬇️', '⬇️', '⬇️']
3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN ['⬇️', '⬇️', '⬇️', '⬇️', '✅', '⬇️', '⬇️', '⬇️']
3DHP-GT-CAM_NO_FACTOR-TEST_TS1_6 ['⬇️', '⬇️', '⬆️', '⬇️', '⬇️', '⬇️', '⬇️', '⬆️']
FIT3D-GT-CAM_NO_FACTOR-ALL_TEST ['⬇️', '⬇️', '⬇️', '⬇️', '⬇️']
FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_FIXED_DIST_5-ALL_TEST ['✅', '✅']
FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-ALL_TEST ['✅', '⬇️', '✅', '✅', '✅']
FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TR_S03 ['⬇️', '⬇️']
FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TS_S4710 ['✅', '✅']
FIT3D-GT-CAM_NO_FACTOR-TR_S03 ['⬇️']
FIT3D-GT-CAM_NO_FACTO

In [8]:
def update_mpjpe(page_id, mpjpe, p_mpjpe):
    url = f"https://api.notion.com/v1/pages/{page_id}"
    data = {
        "properties": {
            "MPJPE": {
                "number": mpjpe
            },
            "P-MPJPE": {
                "number": p_mpjpe
            }
        },
    }

    response = requests.patch(url, headers=headers, json=data)
    return response.json()

In [9]:
for checkpoint in result_dict.keys():
    if 'cam_no_factor' not in checkpoint: continue
    print(f'\n{checkpoint}')
    model_ids = model_dict[checkpoint]
    for model_id in model_ids:
        page = pages_dict[model_id]
        train = page['properties']['Train dataset']['select'] 
        test = page['properties']['Test dataset']['select']
        # filtering - parent
        if test == None: continue # parent
        
        # get child page's info
        child_page = pages_dict[model_id]
        child_properties = child_page['properties']
        if len(child_properties['Name']['title']) == 0: continue
        child_title = child_properties['Name']['title'][0]['text']['content']
        child_mpjpe = child_properties['MPJPE']['number']
        child_p_mpjpe = child_properties['P-MPJPE']['number']
        if child_mpjpe != None and child_p_mpjpe != None: continue
        if child_title in result_dict[checkpoint].keys():
            e1 = result_dict[checkpoint][child_title]["e1"]
            e2 = result_dict[checkpoint][child_title]["e2"]
            if e1 == None: e1 = -1
            if e2 == None: e2 = -1
            update_mpjpe(model_id, float(f'{e1:.2f}'), float(f'{e2:.2f}'))
            


MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_same_z_ts_s4710

MB_train_fit3d_gt_cam_no_factor_with_canonical2_tr_s03

MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_tr_s03

MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_tr_s1_ts_s5678

MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_ts_s4710

MB_train_h36m_gt_cam_no_factor

MB_train_fit3d_gt_cam_no_factor_ts_s4710

MB_train_h36m_gt_cam_no_factor_with_canonical2_tr_s1_ts_s5678

MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_fixed_dist_5_input_centering

MB_train_h36m_gt_cam_no_factor_tr_s1_ts_s5678

MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_same_z_tr_s03

MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_fixed_dist_5_input_centering_tr_s1_ts_s5678

MB_train_h36m_gt_cam_no_factor_s15678_tr_54138969_ts_others

MB_train_fit3d_gt_cam_no_factor_tr_s03

MB_train_h36m_gt_cam_no_factor_with_canonical2

MB_train_h3